In [1]:
def detect_drift_iks(df_train, df_test):

  iks_statistics = [] # collect statistics generated by IKS
  iks = IKS() # group 0 = reference; group 1 = sliding

  sliding = deque()

  for _, row in df_train.iterrows():
    iks.Add((row, random()), 0)
    wrnd = (row, random()) # we only need to keep RND component for values in the sliding window
    iks.Add(wrnd, 1)
    sliding.append(wrnd)

  # process sliding window
  for _, row in df_test.iterrows():
    iks.Remove(sliding.popleft(), 1)
    wrnd = (row, random())
    iks.Add(wrnd, 1)
    sliding.append(wrnd)

    iks_statistics.append(iks.KS())
  
  return iks_statistics
